## Device use T4 * 2

#### Clone WMamba & U-Mamba Repo

In [1]:
!git clone https://github.com/Yui-Arthur/WMamba.git > /dev/null
%cd WMamba
!git clone https://github.com/bowang-lab/U-Mamba.git > /dev/null

Cloning into 'WMamba'...
remote: Enumerating objects: 223, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 223 (delta 67), reused 58 (delta 58), pack-reused 133
Receiving objects: 100% (223/223), 158.00 KiB | 2.16 MiB/s, done.
Resolving deltas: 100% (122/122), done.
/kaggle/working/WMamba
Cloning into 'U-Mamba'...
remote: Enumerating objects: 301, done.
remote: Counting objects: 100% (60/60), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 301 (delta 46), reused 15 (delta 15), pack-reused 241
Receiving objects: 100% (301/301), 2.91 MiB | 14.26 MiB/s, done.
Resolving deltas: 100% (81/81), done.


In [2]:
!git checkout Kaggle

Branch 'Kaggle' set up to track remote branch 'Kaggle' from 'origin'.
Switched to a new branch 'Kaggle'


#### Mabma & U-Mamba & WMamba reqirement

In [3]:
!pip install causal-conv1d>=1.2.0 --no-cache-dir > /dev/null
!pip install mamba-ssm --no-cache-dir > /dev/null
!cd U-Mamba/umamba && (pip install -e .) > /dev/null

In [4]:
!pip install ptwt >  /dev/null
!pip install PyWavelets >  /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab 4.1.5 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
jupyterlab-lsp 5.1.0 requires jupyter-lsp>=2.0.0, but you have jupyter-lsp 1.5.1 which is incompatible.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
momepy 0.7.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.


In [5]:
!sudo bash setup_wmamba.sh

Setup WMamba
CP File From W-Mamba to U-Mamba
Trainer:
  nnUNetTrainerUMambaEncNoAMP.py
  nnUNetTrainerWMambaBase.py
  nnUNetTrainerUMambaEnc.py
  nnUNetTrainerUMambaBot.py
  nnUNetTrainerWMambaBot.py
Nets:
  WMambaBot_3d.py
Others:
  paths.py
  waveletLayer.py


### set env variable

In [6]:
### DO NOT EDIT !!! ###
nnUNet_preprocessed="/home/nnUNet_preprocessed/"
nnUNet_raw="/home/nnUNet_raw/"
nnUNet_results="/kaggle/working/"

%env ipynb=1
%env nnUNet_preprocessed={nnUNet_preprocessed}
%env nnUNet_raw={nnUNet_raw}
%env nnUNet_results={nnUNet_results}



env: ipynb=1
env: nnUNet_preprocessed=/home/nnUNet_preprocessed/
env: nnUNet_raw=/home/nnUNet_raw/
env: nnUNet_results=/kaggle/working/


### parameter (can modify)

In [7]:
### Epoch ###
# **the maximum epoch include all train epoch in every session** #
%env training_epochs=300

### Checkpoint ###
# every {checkpoint_save_every} epochs will save one checkpoint
%env checkpoint_save_every = 10
load_checkpoint = True

prev_checkpoint_f0 = "/kaggle/input/0603/checkpoint_latest_0.pth"
prev_checkpoint_f1 = "/kaggle/input/0603/checkpoint_latest_1.pth"

### Kaggle Dataset Input Path ###
dataset_path = "/kaggle/input/Dataset110_CTImage.mp4"
### Dataset ID ###
dataset_id = 110

### Set Trainer/Model ###
trainer = "nnUNetTrainerWMambaBot"

### Set GPU DRAM for dataset preproccess ###
gpu_dram = 8


env: training_epochs=300
env: checkpoint_save_every=10


### dataset prepaired

In [8]:
if dataset_id == 110:
    """===== Dataset 110 Covid-19 ===="""
    """ unrar data folder and move to home. """
    !apt install unrar
    !(unrar x {dataset_path} {nnUNet_raw}) >  /dev/null
    """dataset preoproccess about 20 mins."""
    !nnUNetv2_plan_and_preprocess -d 110 --verify_dataset_integrity -c 3d_fullres -gpu_memory_target {gpu_dram} > /dev/null
    dataset_id = 110
    dataset_name = "Dataset110_CTImage"
else:
    """===== Dastaset 4 Hippocampus ===="""
    """ unzip data folder and move to home. """
    !(tar -C  /home/ -xvf {dataset_path}) > /dev/null
    """ dataset preoproccess. """
    !nnUNetv2_convert_MSD_dataset -i /home/Task04_Hippocampus 
    !nnUNetv2_plan_and_preprocess -d 4 --verify_dataset_integrity -c 3d_fullres -gpu_memory_target {gpu_dram} > /dev/null
    dataset_id = 4
    dataset_name = "Dataset004_Hippocampus"





The following NEW packages will be installed:
  unrar
0 upgraded, 1 newly installed, 0 to remove and 51 not upgraded.
Need to get 113 kB of archives.
After this operation, 406 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/multiverse amd64 unrar amd64 1:5.6.6-2build1 [113 kB]
Fetched 113 kB in 0s (613 kB/s)

78Selecting previously unselected package unrar.
(Reading database ... 113807 files and directories currently installed.)
Preparing to unpack .../unrar_1%3a5.6.6-2build1_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 20%] [###########...............................................] 8Unpacking unrar (1:5.6.6-2build1) ...
7Progress: [ 40%] [#######################...................................] 8Setting up unrar (1:5.6.6-2build1) ...
7Progress: [ 60%] [##################################........................] 8update-alternatives: using /usr/bin/unrar-nonfree to pro

### train

In [9]:
# from pathlib import Path
# if not load_checkpoint:
#     ### first training (no checkpoint) ###
#     !CUDA_VISIBLE_DEVICES=0 nnUNetv2_train {dataset_id} 3d_fullres 0 -tr {trainer} --npz & \
#     CUDA_VISIBLE_DEVICES=1 nnUNetv2_train {dataset_id} 3d_fullres 1 -tr {trainer} --npz
# else:
#     ### load checkpoint, continue train
#     current_trainer_folder = f"{nnUNet_results}{dataset_name}/{trainer}__nnUNetPlans__3d_fullres/"

#     !mkdir -p {current_trainer_folder}/fold_0
#     !mkdir -p {current_trainer_folder}/fold_1

#     !cp {prev_checkpoint_f0} {current_trainer_folder}/fold_0/checkpoint_best.pth
#     !cp {prev_checkpoint_f1} {current_trainer_folder}/fold_1/checkpoint_best.pth
    
#     # check the checkpoint can be successful loaded
#     f0 = [i.name for i in Path(current_trainer_folder+"fold_0").glob("*.pth")]
#     f1 = [i.name for i in Path(current_trainer_folder+"fold_1").glob("*.pth")]
#     print("Load Checkpoint fold 0",f0)
#     print("Load Checkpoint fold 1",f1)
#     if len(f0) <1 or len(f1) <1:
#         raise ValueError("Load Checkpoint Failed")
    
#     !CUDA_VISIBLE_DEVICES=0 nnUNetv2_train {dataset_id} 3d_fullres 0 -tr {trainer} --c --npz & \
#     CUDA_VISIBLE_DEVICES=1 nnUNetv2_train {dataset_id} 3d_fullres 1 -tr {trainer} --c  --npz

In [10]:
current_trainer_folder = f"{nnUNet_results}{dataset_name}/{trainer}__nnUNetPlans__3d_fullres/"
!mkdir -p {current_trainer_folder}
!cp -r /kaggle/input/fold_0 {current_trainer_folder}/
!cp -r /kaggle/input/fold_1 {current_trainer_folder}/
!cp -r /kaggle/input/fold_2 {current_trainer_folder}/
!cp -r /kaggle/input/fold_3 {current_trainer_folder}/
!cp -r /kaggle/input/fold_4 {current_trainer_folder}/
!cp /kaggle/input/dataset.json {current_trainer_folder}/
!cp /kaggle/input/dataset_fingerprint.json {current_trainer_folder}/
!cp /kaggle/input/plans.json {current_trainer_folder}/



### find best fold

In [11]:
!nnUNetv2_find_best_configuration {dataset_id} -c 3d_fullres -tr {trainer} -f 0 1 2 3 4

Traceback (most recent call last):
  File "/opt/conda/bin/nnUNetv2_find_best_configuration", line 33, in <module>
    sys.exit(load_entry_point('nnunetv2', 'console_scripts', 'nnUNetv2_find_best_configuration')())
  File "/kaggle/working/WMamba/U-Mamba/umamba/nnunetv2/evaluation/find_best_configuration.py", line 295, in find_best_configuration_entry_point
    find_best_configuration(dataset_name, model_dict, allow_ensembling=not args.disable_ensembling,
  File "/kaggle/working/WMamba/U-Mamba/umamba/nnunetv2/evaluation/find_best_configuration.py", line 100, in find_best_configuration
    accumulate_cv_results(output_folder, merged_output_folder, folds, num_processes, overwrite)
  File "/kaggle/working/WMamba/U-Mamba/umamba/nnunetv2/evaluation/accumulate_cv_results.py", line 50, in accumulate_cv_results
    compute_metrics_on_folder(gt_folder,
  File "/kaggle/working/WMamba/U-Mamba/umamba/nnunetv2/evaluation/evaluate_predictions.py", line 152, in compute_metrics_on_folder
    metric_list

### predict 

In [12]:
!nnUNetv2_predict -i "{nnUNet_raw}{dataset_name}/imagesTs"\
-o "{nnUNet_results}{dataset_name}/predTs" \
-d {dataset_id} -c 3d_fullres -tr {trainer} -f 0 1 2 3 4 > /dev/null

2024-06-09 19:26:05.911192: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-09 19:26:05.911305: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-09 19:26:06.031492: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/kaggle/working/WMamba/U-Mamba/umamba/nnunetv2/waveletLayer.py:30: UserWarning: The wavelet_level is zero, auto set one to avoid error
  warnings.warn("The wavelet_level is zero, auto set one to avoid error")
100%|███████████████████████████████████████████| 27/27 [00:55<00:00,  2.07s/it]


In [13]:
!python ./createSubmissionFile.py -s "{nnUNet_results}{dataset_name}/predTs" -d "{nnUNet_results}/predTs_rename"
!(cd {nnUNet_results} && zip -r pred.zip predTs_rename/)

[01/50] Convert file from /kaggle/working/Dataset110_CTImage/predTs/covid19_0302.nii.gz to /kaggle/working//predTs_rename/302.nii.gz
[02/50] Convert file from /kaggle/working/Dataset110_CTImage/predTs/covid19_0697.nii.gz to /kaggle/working//predTs_rename/697.nii.gz
[03/50] Convert file from /kaggle/working/Dataset110_CTImage/predTs/covid19_0408.nii.gz to /kaggle/working//predTs_rename/408.nii.gz
[04/50] Convert file from /kaggle/working/Dataset110_CTImage/predTs/covid19_0543.nii.gz to /kaggle/working//predTs_rename/543.nii.gz
[05/50] Convert file from /kaggle/working/Dataset110_CTImage/predTs/covid19_0064.nii.gz to /kaggle/working//predTs_rename/064.nii.gz
[06/50] Convert file from /kaggle/working/Dataset110_CTImage/predTs/covid19_0690.nii.gz to /kaggle/working//predTs_rename/690.nii.gz
[07/50] Convert file from /kaggle/working/Dataset110_CTImage/predTs/covid19_0089.nii.gz to /kaggle/working//predTs_rename/089.nii.gz
[08/50] Convert file from /kaggle/working/Dataset110_CTImage/predTs/c